# Implicit ALS

_Adapted from [@sophwatts](https://github.com/sophwats/Implicit-ALS)_

We usually consider using ALS on a set of user/product ratings. But what if the data isn't so self explanatory?

### A day trip to the library
Consider, for example, the data collected by a local library. The library records which users took out each books and how long they kept the books before returning them. 

As such, we have no explicit indication that a user liked or disliked the books they took out - Just because you borrowed a book does not mean that you enjoyed it, or even read it.
Furthermore, the missing data is of interest - the fact that a user has not taken out a specific book could indicate that they dislike that genre, or that they haven't been to that section of the library.

Furthermore the same user action could have many different causes. Suppose you withdraw a book three times. That might indicate that you loved the book, but it may also indicate that the book doesn't appeal to you as strongly as some other books you withdrew so you never got round to reading it the first two times.

To make the situation even worse, implicit data is often dirty. For example, a user may withdraw a library book for their child using their account, or they may accidentally pick up a book that was sitting on the counter. 

### The solution
Based on the standard ALS implementation, [Hu et al. (2008)](http://yifanhu.net/PUB/cf.pdf) presented a methodolgy for carrying out ALS when dealing with implicit data. 

The general idea is that we have some recorded observations $r_{u,i}$ denoting the level of interaction user $u$ had with product $i$. For example, if a user $1$ borrowed book $4$ once we may set $r_{1,4}=1$. Alternatively we may wish to allow $r_{u,i}$ to hold information about how many days the book was borrowed for. (There is a lot of freedom in this set up, so we need to make some data specific decisions regarding how we will select $r_{u,i}$).

Given the set of observations $r_{u,i}$, a binary indicator $p_{u,i}$ is introduced where:

$ p_{u,i} = \begin{cases} 1 & \mbox{if } r_{i,j}>0 \\
0 & \mbox{otherwise.} \end{cases} $


A confidence parameter $\alpha$ lets the user determine how much importance they wish to place on the recorded $r_{u,i}$. This leads to the introduction of $c_{u,i}$ which we take to be the confidence we have in the strength of user $u$'s reaction to product $i$: 
$c_{u,i} = 1 + \alpha r_{u,i}$.

Let $N_u$ denote the number of users, and $N_p$ denote the number of products. Let $k\in \mathbb{R}^+$ be a user defined number of factors. 
Now, in implicit ALS the goal is to find matrices $X\in \mathbb{R}^{N_u \times k}$ and $Y\in \mathbb{R}^{N_p \times k}$ such that the following cost function is minimised:

$\sum_{u,i} c_{u,i}(p_{u,i}-X_u^T Y_i)^2 + \lambda (\sum_u \| X_u\|^2 + \sum_{i} \| y_u\|^2), $


where
$X_u$ is the $u$th row of X, 
$Y_i$ is the $i$th row of Y,
\lambda is a user defined parameter which prevents overfitting. 

With this minimisation at hand, we are able to recover estimates of $c_{u,i}$, and thus of $r_{u,i}$ for interactions which have not yet occured. 

### Let's get going
We are going to run implicit ALS using the implementation given in the pyspark.mllib.recommendation module. 

The data we will be using can be found at http://www2.informatik.uni-freiburg.de/~cziegler/BX/

In [ ]:
#Set up a spark context

import pyspark

spark = (pyspark.sql.SparkSession.builder
         .appName('implicitALS')
         .getOrCreate())
sc = spark.sparkContext

# The Data


In the cell below, we download and unzip the data. The two files we are interested in are BX-Books.csv and BX-Book-Ratings.csv, which follow these schema: 

### BX-Books.csv

| Field Name | Type | Description |
| ---------- | -----| ----------- |
|ISBN |  String | length 10, alphanumeric |
| Book-Title | String | Title of book |
|Book-Author | String| Name of author |
| Year-Of-Publication | String | yyyy|
|Publisher| String |Name of publisher |
|Image-URL-S | String| URL for small image on amazon.com |
|Image-URL-M | String| URL for medium image on amazon.com |
|Image-URL-L | String| URL for large image on amazon.com|


### BX-Book-Ratings.csv
| Field Name | Type | Description |
| ---------- | ---- | ----------- |
|User-ID |  Integer | Range from 2 to 278854 |
| ISBN | String| length 10, alphanumeric |
|Book-Rating| Integer | 1-10 denotes dislike-like. 0 denotes implicit interaction|

In [ ]:
#Downloading and unzipping the data
!wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
!unzip BX-CSV-Dump.zip

The cell above loads three .csv files into the working directory. We are interested in the files "BX-Books-Ratings.csv" and "BX-Books.csv". The first three columns of "BX-Book-Ratings" are the user id, an isbn which identifies the book, and a rating. A '0' in the rating column is used to denote that an implicit interaction occured between the user an the book. It is this data that we are interested in, and we extract such rows using the following grep command:

In [ ]:
!head BX-Book-Ratings.csv

In [ ]:
!echo '"user_id";"isbn";"observation"' > implicit.csv

In [ ]:
!grep '"0"' BX-Book-Ratings.csv >> implicit.csv

In [ ]:
!head implicit.csv

In [ ]:
#Load in the data
ratings_df = spark.read.csv('implicit.csv', sep=';', header=True, inferSchema=True)

Let's have a look at the first 10 entries in the ratings file: 

In [ ]:
ratings_df.show(10)

The implicit ALS function we are going to use requires that product ids are integers. At the moment we have unique ISBNs, which contain a mixture of numbers and letters, so we must convert to integers. This can be done using the zipWithIndex() function which takes an RDD and joins unique ids to each entry. 

In [ ]:
ratings_df.createGlobalTempView("ratings")


In [ ]:
ratings_df = spark.sql("""
SELECT
  user_id
, dense_rank() OVER (ORDER BY isbn) AS isbn_id
, isbn
-- we want 1, not 0, as having interacted with a book
, 1 as rating
FROM global_temp.ratings""")
ratings_df.show(10)

We now import the ALS function from the mllib module, and build the model. 

In [ ]:
from pyspark.ml.recommendation import ALS
als = ALS(rank=5, maxIter=5, alpha=0.99, implicitPrefs=True,
          userCol="user_id", itemCol="isbn_id", ratingCol="rating",
          nonnegative=True)

In [ ]:
training, test = ratings_df.randomSplit([0.8, 0.2])

model = als.fit(training)

In [ ]:
#Using the predict all function to give predictions for any unseens. 
predictions = model.transform(test)

We can now look at predictions for a range of user, product pairs:

In [ ]:
predictions.take(10)

We can use `.filter()` and `.orderBy()` to view the 20 highest rated items for that user. 

In [ ]:
(predictions.filter(predictions.user_id == 8)
 .orderBy("prediction", ascending=False).take(10))

The .recommendForAllItems function allows us to view predicted ratings for specific user, item pairs. 

In [ ]:
model.recommendForAllItems(8).take(10)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

### Conclusion 
In this notebook we saw how to build a basic implicit ALS model in Spark. However, the data used was fairly plain, with "0"s being used for all implicit interactions. Furtherwork should consider a dataset more suited to implicit ALS. 